<a href="https://colab.research.google.com/github/truyentdm/NotebookTTS/blob/main/VietTTS_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Quick start

In [1]:
%cd /content
!git clone https://github.com/NTT123/vietTTS.git
%cd /content/vietTTS
!pip3 install -e .
!bash ./scripts/quick_start.sh
from google.colab import output
output.clear()

In [2]:
text = """
xin chào, Đoạn trường tân thanh, thường được biết đến với cái tên đơn giản là Truyện Kiều, là một truyện thơ của đại thi hào Nguyễn Du.
Đây được xem là truyện thơ nổi tiếng nhất và xét vào hàng kinh điển trong văn học Việt Nam.
Câu chuyện dựa theo tiểu thuyết Kim Vân Kiều truyện của Thanh Tâm Tài Nhân, một thi sĩ thời nhà Minh, Trung Quốc.
Tác phẩm kể lại cuộc đời, những thử thách và đau khổ của Thúy Kiều, một phụ nữ trẻ xinh đẹp và tài năng, phải hy sinh thân mình để cứu gia đình.
Để cứu cha và em trai khỏi tù, cô bán mình kết hôn với một người đàn ông trung niên, không biết rằng anh ta là một kẻ buôn người, và bị ép làm kĩ nữ trong lầu xanh.
"""
!python3 -m vietTTS.synthesizer --lexicon-file assets/infore/lexicon.txt --text="{text}" --output=clip.wav --silence-duration 0.2
import IPython.display as ipd
ipd.Audio('clip.wav')

Normalized text input: xin chào sil đoạn trường tân thanh sil thường được biết đến với cái tên đơn giản là truyện kiều sil là một truyện thơ của đại thi hào nguyễn du sil đây được xem là truyện thơ nổi tiếng nhất và xét vào hàng kinh điển trong văn học việt nam sil câu chuyện dựa theo tiểu thuyết kim vân kiều truyện của thanh tâm tài nhân sil một thi sĩ thời nhà minh sil trung quốc sil tác phẩm kể lại cuộc đời sil những thử thách và đau khổ của thúy kiều sil một phụ nữ trẻ xinh đẹp và tài năng sil phải hy sinh thân mình để cứu gia đình sil để cứu cha và em trai khỏi tù sil cô bán mình kết hôn với một người đàn ông trung niên sil không biết rằng anh ta là một kẻ buôn người sil và bị ép làm kĩ nữ trong lầu xanh sil

writing output to file clip.wav


# Train duration model and acoustic model

> Văn bản được thụt lề



In [3]:
!bash ./scripts/download_aligned_infore_dataset.sh

bash: ./scripts/download_aligned_infore_dataset.sh: No such file or directory


In [4]:
!python3 -m vietTTS.nat.duration_trainer

---------------------------  -------------------------
duration_lstm_dim            256
vocab_size                   256
duration_embed_dropout_rate  0.5
num_training_steps           200000
postnet_dim                  512
acoustic_decoder_dim         512
acoustic_encoder_dim         256
max_phoneme_seq_len          256
max_wave_len                 196608
special_phonemes             ['sil', 'sp', 'spn', ' ']
sil_index                    0
sp_index                     0
word_end_index               3
mel_dim                      80
n_fft                        1024
sample_rate                  16000
fmin                         0.0
fmax                         8000
batch_size                   64
learning_rate                0.0001
duration_learning_rate       0.0001
max_grad_norm                1.0
weight_decay                 0.0001
token_mask_prob              0.1
ckpt_dir                     assets/infore/nat
data_dir                     train_data
---------------------------  ----

In [ ]:
!python3 -m vietTTS.nat.acoustic_trainer

---------------------------  -------------------------
duration_lstm_dim            256
vocab_size                   256
duration_embed_dropout_rate  0.5
num_training_steps           200000
postnet_dim                  512
acoustic_decoder_dim         512
acoustic_encoder_dim         256
max_phoneme_seq_len          384
max_wave_len                 196608
special_phonemes             ['sil', 'sp', 'spn', ' ']
sil_index                    0
sp_index                     1
word_end_index               3
mel_dim                      80
n_fft                        1024
sample_rate                  16000
fmin                         0.0
fmax                         8000
batch_size                   64
learning_rate                0.0001
duration_learning_rate       0.0001
max_grad_norm                1.0
weight_decay                 0.0001
ckpt_dir                     assets/infore/nat
data_dir                     train_data
---------------------------  -------------------------
Resuming fr

# Train HiFiGAN vocoder

In [5]:
%cd /content/vietTTS
!bash ./scripts/download_aligned_infore_dataset.sh


/content/vietTTS
bash: ./scripts/download_aligned_infore_dataset.sh: No such file or directory


In [6]:
!git clone https://github.com/jik876/hifi-gan.git
%cd hifi-gan
!ln -sf /content/vietTTS/train_data /content/vietTTS/hifi-gan/data
!cd data;  ls -1 *.TextGrid| sed -e 's/\.TextGrid$//' > /content/vietTTS/hifi-gan/files.txt
!head -n 100 ./files.txt > val_files.txt
!tail -n +101 files.txt > train_files.txt
!cp /content/vietTTS/assets/hifigan/config.json ./config.json

Cloning into 'hifi-gan'...
remote: Enumerating objects: 48, done.
remote: Total 48 (delta 0), reused 0 (delta 0), pack-reused 48
Receiving objects: 100% (48/48), 620.94 KiB | 18.26 MiB/s, done.
Resolving deltas: 100% (20/20), done.
/content/vietTTS/hifi-gan
/bin/bash: line 1: cd: data: No such file or directory
ls: cannot access '*.TextGrid': No such file or directory


In [7]:
!python3 train.py --config config.json --input_wavs_dir=data  --input_training_file=train_files.txt  --input_validation_file=val_files.txt

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Time taken for epoch 1434 is 0 sec

Epoch: 1435
Time taken for epoch 1435 is 0 sec

Epoch: 1436
Time taken for epoch 1436 is 0 sec

Epoch: 1437
Time taken for epoch 1437 is 0 sec

Epoch: 1438
Time taken for epoch 1438 is 0 sec

Epoch: 1439
Time taken for epoch 1439 is 0 sec

Epoch: 1440
Time taken for epoch 1440 is 0 sec

Epoch: 1441
Time taken for epoch 1441 is 0 sec

Epoch: 1442
Time taken for epoch 1442 is 0 sec

Epoch: 1443
Time taken for epoch 1443 is 0 sec

Epoch: 1444
Time taken for epoch 1444 is 0 sec

Epoch: 1445
Time taken for epoch 1445 is 0 sec

Epoch: 1446
Time taken for epoch 1446 is 0 sec

Epoch: 1447
Time taken for epoch 1447 is 0 sec

Epoch: 1448
Time taken for epoch 1448 is 0 sec

Epoch: 1449
Time taken for epoch 1449 is 0 sec

Epoch: 1450
Time taken for epoch 1450 is 0 sec

Epoch: 1451
Time taken for epoch 1451 is 0 sec

Epoch: 1452
Time taken for epoch 1452 is 0 sec

Epoch: 1453
Time taken for epoch 1453 is 0 